# N-Gram Language Models #

Language models are one of the most important concepts in Natural Language Processing (NLP). They help us assign probabilities to sequence of words, such as sentences or phrases. Their application spans beyond NLP. For example, language models help us obtain better speech recognition, optical character recognition (OCR) and information retrieval results, to name a few. Given a sentence $S$ with a set of $w_i$ words, $i=1,2,...,n$, language models formally define the probability of the sentence as the probability of having the particular sequence of words:  
$$ \Large p(S)=p(w_1, w_2, w_3, ... , w_n) $$  

They also help us predict the probability of a specific word given the previous words in the sentence:  

$$ \Large p(w | w_{-1}, w_{-2}..w_{-k}) $$  

The probability of a sentence is computed using the chain rule:  

$$ \Large p(S)=p(w_1, w_2, w_3, ... , w_n) = \prod_i^n {p(w_i|w_1,w_2,...,w_{i-1})} $$  

When computing this probability we use the Markov assumption which simplifies the computation of the above probability:    

$$ \Large p(S)=p(w_1, w_2, w_3, ... , w_n) \approx \prod_i^n {p(w_i|w_{i-k},w_{i-(k-1)},...,w_{i-1})} $$  

The Markov assumption states that the conditional probability of a word $w_i$, given all of the previous words in the sentence, could be approximated by considering only the $k$ previous words.


### N-Gram Models ###

#### Unigram Model ####
The unigram language model is the simplest of the n-gram models. Under this model the probability of the sentence $S$ is simply a product of the probabilities of each individual word in the sentence:  

$$ \Large p(S) \approx \prod_i^n {p(w_i)} $$  

N-gram probabilities are computed using Maximum Likelihood Estimates (MLE). For the unigram language model we compute MLE by counting the number of times word $w_i$ occurred in the collection and we divide that number by the total number of words $v$ in the collection:
$$ \Large p(w_i) = \frac{count(w_i)}{\sum_{i=1}^{v}{count(w_i)}} $$  

#### Bigram Model ####
In the bigram model the probability of a word is conditioned only on the previous word:  
$$ \Large p(S) \approx \prod_i^n {p(w_i|w_{i-1})} $$  

The MLE for the bigram LM is computed by dividing the number of times words $w_i$ and $w_{i-1}$ occured together in the collection with the number of occurances of word $w_i$, $count(w_i)$ :

$$ \Large p(w_i|w_{i-1}) = \frac{count(w_i|w_{i-1})}{count(w_i)} $$  

#### Trigram Model ####
In the trigram model the probability of a word is conditioned on the previous two words:  
$$ \Large p(S) \approx \prod_i^n {p(w_i|w_{i-1},w_{i-2})} $$  

The MLE for the trigram LM is computed by dividing the number of times words $w_i$, $w_{i-1}$, $w_{i-2}$ occurred together in the collection with the count for $w_{i-1}$ and $w_{i-2}$ occurring together, $count(w_{i-1},w_{i-2})$:

$$ \Large p(w_i|w_{i-1},w_{i-2}) = \frac{count(w_i|w_{i-1},w_{i-2})}{count(w_{i-1},w_{i-2})} $$  

## Example ##
In this task we are going to compute bigram and trigram language models using a novel. Rather than implementing language models on our own we are going to use the nltk package.  

Look into the books folder and choose one of the ten books that you would like to compute language models over. In the example below we are using the novel "Siddhartha" by Herman Hesse. Load the book and extract words and sentences:

In [ ]:
import nltk
import itertools
stopwords_list = nltk.corpus.stopwords.words('english')
book= open('./books/hesse_siddhartha.txt','r').read()

words = nltk.word_tokenize(book)
words = [word.lower() for word in words]
sentences = nltk.sent_tokenize(book)
tokenized_sentences = []
for sentence in sentences:
	words = nltk.word_tokenize(sentence)
	words = [word.lower() for word in words]
	tokenized_sentences.append(words)

Compute bigram language model:

In [ ]:
#First compute the bigrams (i.e. the tuple of words that occur together)
bigram_model  = nltk.bigrams(words)
#Then compute the frequency for each bigram
bigram_frequency_word = nltk.ConditionalFreqDist(bigram_model)
#To get the bigram probabilities we would need to normalize the bigram frequencies:
bigram_probability_word = nltk.ConditionalProbDist(bigram_frequency_word, nltk.MLEProbDist)

#Let's compute the bigram probabilities constraining on sentences:
bigram_frequency_sent = nltk.ConditionalFreqDist((word[0],word[1]) for word in list( itertools.chain (*[nltk.bigrams(i) for i in tokenized_sentences])))
bigram_probability_sent = nltk.ConditionalProbDist(bigram_frequency_sent, nltk.MLEProbDist)

#Let's print the bigram probabilities computed over sentences:
bigram_probability = bigram_probability_sent
bigram_frequency = bigram_frequency_sent

all_bigrams2 = {}
for source_word in bigram_probability:
	prob_words = bigram_probability[source_word].samples()
	denom = len(prob_words)
	all_bigrams2[source_word]={}
	for target_word in prob_words:
		prob = bigram_probability[source_word].prob(target_word)
		all_bigrams2[source_word][target_word] = prob
		print ("p("+target_word+"|"+source_word+")={0:.4f}".format(prob))

In [ ]:
#We could also obtain bigram probabilities for specific words:
bigram_probability.conditions()
query = bigram_probability["suffering"].prob("was")
print (str(query))

Compute trigram language model:

In [ ]:
#First compute the trigrams:
trigram_model  = nltk.trigrams(words)
#Then compute the frequency for each trigram:
trigram_words = (((word[0], word[1]), word[2]) for word in trigram_model)
trigram_frequency_word = nltk.ConditionalFreqDist(trigram_words)
#To get the trigram probabilities we would need to normalize the trigram frequencies:
trigram_probability_word = nltk.ConditionalProbDist(trigram_frequency_word, nltk.MLEProbDist)
#Let's compute the trigram probabilities constraining on sentences:
trigram_frequency_sent = nltk.ConditionalFreqDist(((word[0], word[1]), word[2])  for word in list( itertools.chain (*[nltk.trigrams(i) for i in tokenized_sentences])))
trigram_probability_sent = nltk.ConditionalProbDist(trigram_frequency_sent, nltk.MLEProbDist)

#Let's print the trigram probabilities computed over sentences:
trigram_probability = trigram_probability_sent
trigram_frequency = trigram_frequency_sent
all_trigrams2 = {}
for source_word in trigram_probability:
	prob_words = trigram_probability[source_word].samples()
	denom = len(prob_words)
	all_trigrams2[source_word]={}
	for target_word in prob_words:
		prob = trigram_probability[source_word].prob(target_word)
		freq = trigram_frequency[source_word][target_word]
		prob2 = (1.0)*freq/denom
		all_trigrams2[source_word][target_word] = prob
		print ("p("+target_word+"|"+source_word[0]+","+source_word[1]+")={0:.4f}".format(prob))

**[Assignment 1]** What about unigram language model? Using the above example code for computing bigram and trigram language models seen if you could implement the unigram language model.

**[Solution 1]**

In [ ]:
#Enter your code here.